# Reviews Sentiment Analysis By Fine Tuning Bert

## Install packages

In [14]:
%pip install datasets
%pip install transformers[torch]
%pip install -q peft
%pip install -q evaluate
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.5.0-cp311-cp311-win_amd64.whl.metadata (11 kB)
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.8 kB 2.7 MB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 814.9 kB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.0-cp311-cp311-win_amd64.whl (11.0 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
   ---------------------------------------- 0.0/44.7 MB ? eta -:--:--
   ---------------------------------------- 0.2/44.7 MB 6.1 MB/s eta 0:00:08
   -------------------------

# Preparation

Import all necessary libraries

In [2]:
import torch

import numpy as np

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from peft import LoraConfig, TaskType, get_peft_model, PeftModel

import evaluate

e:\Code\Project\Sentiment_Analysis_Bert_Fine_Tuning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checking avaiable devices

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cpu


## Data preprocessing

Load dataset. We use the IMDB dataset.

In [4]:
raw_datasets = load_dataset("imdb")

Tokenizer from BERT

In [5]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Test the tokenizer 

In [6]:
sentence = "Good morning, today is a good day"
input = tokenizer(sentence, padding="max_length", truncation=True)
print(input)

{'input_ids': [101, 2750, 2106, 117, 2052, 1110, 170, 1363, 1285, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Prepare tokenized dataset from the IMDB dataset

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

full_train_dataset = tokenized_datasets["train"].shuffle(seed=77)
full_eval_dataset = tokenized_datasets["test"].shuffle(seed=77).select(range(2500))   #Small sample size for testing 

print(len(tokenized_datasets["train"]))
print(len(tokenized_datasets["test"]))

25000
25000


## Model and Training

LoRA (Low Rank Adaptation) is applied to fine tune BERT model

Configuration for LoRA. 
- SEQ and CLS are used for training BERT model
- Rank is set to 1 for training (larger rank does not make much difference)


In [8]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

Define pretrained BERT model

In [9]:

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


File paths definitions 
- `model_path` : Model path for loading to train
- `save_path` : Model path for saving after training

In [11]:
model_path = "./fine_tuned_model"
save_path = "./fine_tuned_model_1"

In [12]:

base_model= AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
merged_model= PeftModel.from_pretrained(base_model, model_path)
merged_model= merged_model.merge_and_unload()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Metric for evalation. We use sklearn's metric

In [15]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=-1)
    return metric.compute(predictions=pred, references=labels)

Training Arguments

In [18]:
training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch",
                                  num_train_epochs = 5,
                                  per_device_train_batch_size = 32,
                                  per_device_eval_batch_size = 32
                                  )

Define a trainer

In [21]:
trainer = Trainer(
    model=model.to(device),
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset,
    compute_metrics=compute_metrics,
)

Start training

In [23]:
trainer.evaluate()

KeyboardInterrupt: 

Save fine tuned model

In [ ]:
trainer.model.save_pretrained(save_path)

## Load a fine tuned model

In [20]:
load_path = "./fine_tuned_model"
trainer.model.save_pretrained(load_path)

In [ ]:
from peft import PeftModel

base_model= AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# merge base model and fine-tuned model
merged_model= PeftModel.from_pretrained(base_model, trainer_filepath)
merged_model= merged_model.merge_and_unload()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: Can't find 'adapter_config.json' at '/content/drive/MyDrive/bert_sentiment/train_2'